参考：

https://github.com/small-yellow-duck/facebook_auction/blob/master/fba-sub9.py

https://github.com/wallinm1/kaggle-facebook-bot/blob/master/facebook_notebook.ipynb


In [1]:
# import 一些库
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# 加载数据

train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
test['outcome'] = -1
bids = pd.read_csv('input/bids.csv').sort_values(['auction', 'time'])
bot_or_human = pd.concat([ train, test ])
del test
del train

In [3]:
## 去掉账户和地址的后5个字母
def address_strip(x):
    return x[0:-5]
bot_or_human.address = bot_or_human.address.apply(address_strip)
bot_or_human.payment_account = bot_or_human.payment_account.apply(address_strip)
# 特征提取：账户与地址是否一致
bot_or_human['payment_address_same'] = bot_or_human['address']==bot_or_human['payment_account']
bot_or_human['payment_address_same']= bot_or_human['payment_address_same'].astype(int)

In [4]:
# log熵计算

def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))
    return e



# 特征提取：每个拍卖者的总出价次数
b = bids.groupby(['bidder_id']).size().reset_index().rename(columns = {0:'bids_count'}).fillna(0)
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')
del b 
# 特征提取：每个拍卖者参加拍卖场数
# 参考 http://stackoverflow.com/questions/15411158/pandas-countdistinct-equivalent
b = bids.groupby(['bidder_id']).auction.nunique().reset_index().rename(columns = {'auction':'auctions_count'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')
del b 

# 特征提取：拍卖者每场拍卖bid数的中位数
a = bids.groupby(['bidder_id', 'auction']).size().reset_index()
b = a.groupby('bidder_id')[0].median().reset_index().rename(columns = {0:'bids_per_auction_median'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')
del a
del b 
#  拍卖者登录country总数
b = bids.groupby(['bidder_id']).country.nunique().reset_index().rename(columns = {'country':'countries_count'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')

#  拍卖者登录国家熵
b = bids.groupby(['bidder_id', 'country']).size().reset_index()
b= b.rename(columns = {0:'country_count'})
c = b.groupby('bidder_id').country_count.apply(log_entropy).reset_index()
c= c.rename(columns = {'country_count':'country_entropy'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')
del b,c



#  拍卖者每场拍卖登录country数的中位数平均数与最大值
b = bids[['bidder_id', 'auction', 'country']].groupby(['bidder_id','auction']).country.nunique().reset_index()
b= b.rename(columns = {'country':'countries_per_bidder_per_auction'})

c = b.groupby('bidder_id').countries_per_bidder_per_auction.median().reset_index()
c= c.rename(columns = {'countries_per_bidder_per_auction':'countries_auction_median'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').countries_per_bidder_per_auction.mean().reset_index()
c= c.rename(columns = {'countries_per_bidder_per_auction':'countries_auction_mean'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').countries_per_bidder_per_auction.max().reset_index()
c= c.rename(columns = {'countries_per_bidder_per_auction':'countries_auction_max'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

/usr/local/lib/python2.7/dist-packages/pandas/core/groupby.py:2930: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  inc = np.r_[1, val[1:] != val[:-1]]


In [5]:
#  拍卖者登录ip总数
b = bids.groupby(['bidder_id']).ip.nunique().reset_index().rename(columns = {'ip':'ips_count'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')
del b 
#  拍卖者登录ip熵
b = bids.groupby(['bidder_id', 'ip']).size().reset_index()
b= b.rename(columns = {0:'ip_count'})
c = b.groupby('bidder_id').ip_count.apply(log_entropy).reset_index()
c= c.rename(columns = {'ip_count':'ip_entropy'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')
del b,c



#  拍卖者每场拍卖登录ip数的中位数平均数与最大值
b = bids[['bidder_id', 'auction', 'ip']].groupby(['bidder_id','auction']).ip.nunique().reset_index()
b= b.rename(columns = {'ip':'ips_per_bidder_per_auction'})

c = b.groupby('bidder_id').ips_per_bidder_per_auction.median().reset_index()
c= c.rename(columns = {'ips_per_bidder_per_auction':'ip_auction_median'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').ips_per_bidder_per_auction.mean().reset_index()
c= c.rename(columns = {'ips_per_bidder_per_auction':'ip_auction_mean'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').ips_per_bidder_per_auction.max().reset_index()
c= c.rename(columns = {'ips_per_bidder_per_auction':'ip_auction_max'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

#  拍卖者登录url总数
b = bids.groupby(['bidder_id']).url.nunique().reset_index().rename(columns = {'url':'url_count'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')

#  拍卖者登录url熵
b = bids.groupby(['bidder_id', 'url']).size().reset_index()
b= b.rename(columns = {0:'url_count'})
c = b.groupby('bidder_id').url_count.apply(log_entropy).reset_index()
c= c.rename(columns = {'url_count':'url_entropy'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')
del b,c


#  拍卖者每场拍卖登录url数的中位数平均数与最大值
b = bids[['bidder_id', 'auction', 'url']].groupby(['bidder_id','auction']).url.nunique().reset_index()
b= b.rename(columns = {'url':'urls_per_bidder_per_auction'})

c = b.groupby('bidder_id').urls_per_bidder_per_auction.median().reset_index()
c= c.rename(columns = {'urls_per_bidder_per_auction':'url_auction_median'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').urls_per_bidder_per_auction.mean().reset_index()
c= c.rename(columns = {'urls_per_bidder_per_auction':'url_auction_mean'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').urls_per_bidder_per_auction.max().reset_index()
c= c.rename(columns = {'urls_per_bidder_per_auction':'url_auction_max'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

#  拍卖者登录设备总数
b = bids.groupby(['bidder_id']).device.nunique().reset_index().rename(columns = {'device':'device_count'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')

#  拍卖者登录设备熵
b = bids.groupby(['bidder_id', 'device']).size().reset_index()
b= b.rename(columns = {0:'device_count'})
c = b.groupby('bidder_id').device_count.apply(log_entropy).reset_index()
c= c.rename(columns = {'device_count':'device_entropy'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')
del b,c

#  拍卖者每场拍卖登录设备数的中位数平均数与最大值
b = bids[['bidder_id', 'auction', 'device']].groupby(['bidder_id','auction']).device.nunique().reset_index()
b= b.rename(columns = {'device':'devices_per_bidder_per_auction'})

c = b.groupby('bidder_id').devices_per_bidder_per_auction.median().reset_index()
c= c.rename(columns = {'devices_per_bidder_per_auction':'device_auction_median'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').devices_per_bidder_per_auction.mean().reset_index()
c= c.rename(columns = {'devices_per_bidder_per_auction':'device_auction_mean'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')

c = b.groupby('bidder_id').devices_per_bidder_per_auction.max().reset_index()
c= c.rename(columns = {'devices_per_bidder_per_auction':'device_auction_max'})
bot_or_human = pd.merge(bot_or_human, c, on='bidder_id', how='left')


In [6]:
# 时间单位校正
one_day = 4547368124071.8799
half_hour = one_day/48.0

# 各场拍卖的开始时间
times1 = bids.groupby('auction').time.min().reset_index().rename(columns = {'time':'startt'})
# 结束时间
times2 = bids.groupby('auction').time.max().reset_index().rename(columns = {'time':'endt'})
#  拍卖时长
times = pd.merge(times1, times2, on='auction', how='left')
times['duration'] = times.endt - times.startt

bids = pd.merge(bids, times[['auction', 'startt', 'endt']], on='auction', how='left')

# 拍卖剩余时间与拍卖已经持续时间
bids['t_until_end'] = bids.endt - bids.time
bids['t_since_start'] = bids.time - bids.startt

# 特征提取：拍卖结束前半小时出价数及占总出价数的比例
b = bids[bids['t_until_end'] <= half_hour].groupby(['bidder_id']).size().reset_index().rename(columns = {0:'bids_count_lasthalfh'})
bot_or_human = pd.merge(bot_or_human, b, on='bidder_id', how='left')
bot_or_human['fraction_lasthalfhour'] = bot_or_human['bids_count_lasthalfh']/bot_or_human.bids_count

bot_or_human['fraction_lasthalfhour'].fillna(0, inplace=True)


# 特征提取：出价时间离拍卖开始和结束时间的中位数
b1 = bids.groupby('bidder_id').t_until_end.median().reset_index().rename(columns = {'t_until_end':'t_until_end_median'})
b2 = bids.groupby('bidder_id').t_since_start.median().reset_index().rename(columns = {'t_since_start':'t_since_start_median'})
bot_or_human = pd.merge(bot_or_human, b1, on='bidder_id', how='left')
bot_or_human = pd.merge(bot_or_human, b2, on='bidder_id', how='left')

In [7]:
# 特征提取：同一场内与前一个其他出价者出价的时间间隔的最小值，平均数，标准差

b = bids.sort_values(['auction', 'time'])[['auction', 'time', 'bidder_id']]
# 上一次出价id
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]), index=b.index)
# 上一次出价所属拍卖
b['auction_prev'] = pd.Series(np.append([np.nan], b.auction.values[0:-1]), index=b.index)
# 上一次出价时间
dt = np.append([np.nan], b.time.values[0:-1])
b['time_prev'] = pd.Series(dt, index=b.index)
# 时间间隔
b['dt'] = b['time'] - b['time_prev']
# 排除不在同一场次及同一场次的同一人出价
c = b[b.bidder_id != b.bidder_id_prev][b.auction == b.auction_prev]
# 参考 http://pandas.pydata.org/pandas-docs/stable/groupby.html
d = c.groupby('bidder_id').dt.agg({'dt_min1':'min','dt_std1': 'std', 'dt_mean1': 'mean'}).reset_index()

bot_or_human = pd.merge(bot_or_human, d[['bidder_id','dt_min1','dt_std1', 'dt_mean1']], on='bidder_id', how='left')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [8]:
# 特征提取：与自己前一次出价的时间间隔的最小值，平均数，标准差
 
b = bids.sort_values(['bidder_id', 'time'])[['bidder_id', 'time']]
b['bidder_id_prev'] = pd.Series(np.append([np.nan], b.bidder_id.values[0:-1]), index=b.index)
dt = np.append([np.nan], b.time.values[0:-1])
b['time_prev'] = pd.Series(dt, index=b.index)
b['dt'] = b['time'] - b['time_prev']
c = b[b.bidder_id == b.bidder_id_prev]
d = c.groupby('bidder_id').dt.agg({'dt_min2':'min','dt_std2': 'std', 'dt_mean2': 'mean'}).reset_index()
bot_or_human = pd.merge(bot_or_human, d[['bidder_id','dt_min2','dt_std2', 'dt_mean2']], on='bidder_id', how='left')

In [9]:
# 保存数据

bot_or_human.to_pickle('input/features.pkl')